In [1]:
from mysql.connector import connect

In [26]:
ketnoi = connect(host='localhost', user='root', password='', database='productshipping', port='3308')

In [27]:
cursor = ketnoi.cursor()

## Thực hiện truy vấn bằng Execute

In [6]:
lenh = 'Select * from products where SupplierId = 3'

In [17]:
#thực thi câu lệnh
results = cursor.execute(lenh)

In [8]:
#câu lệnh đơn --> Kết quả được lưu trong Cursor
#cách 1: dùng vòng lặp for duyệt từng dòng trong cursor
for row in cursor:
    print(row)

(6, 'Product VAIIV', 3, 2, 25.0, 0)
(7, 'Product HMLNI', 3, 7, 30.0, 0)
(8, 'Product WVJFP', 3, 2, 40.0, 0)


In [11]:
#cách 2: dùng vòng lặp while kết hợp fetchone -> đọc từng dòng
row = cursor.fetchone()
while row:
    print(row)
    row = cursor.fetchone()

(6, 'Product VAIIV', 3, 2, 25.0, 0)
(7, 'Product HMLNI', 3, 7, 30.0, 0)
(8, 'Product WVJFP', 3, 2, 40.0, 0)


In [14]:
#cách 3: dùng fetchmany(n) để duyệt n dòng cùng 1 lúc, nếu không khai báo n thì mặc định n=1
dong1 = cursor.fetchmany()
dong2_3 = cursor.fetchmany(2)
print(dong1)
print(dong2_3)

[(6, 'Product VAIIV', 3, 2, 25.0, 0)]
[(7, 'Product HMLNI', 3, 7, 30.0, 0), (8, 'Product WVJFP', 3, 2, 40.0, 0)]


In [18]:
#cách 4: dùng fetchall để đọc tất cả các dòng từ vị trí hiện tại của cursor
dong1 = cursor.fetchone()
dong_all = cursor.fetchall()
print(dong_all)

[(7, 'Product HMLNI', 3, 7, 30.0, 0), (8, 'Product WVJFP', 3, 2, 40.0, 0)]


## Thực hiện truy vấn có tham số

In [20]:
#trong MySQL sử dụng ký hiệu %s để truyền tham số cho truy vấn
def doc_san_pham_theo_nha_cung_cap(id_nha_cung_cap, cursor):
    lenh = 'Select * from products where SupplierId = %s'
    #Tạo bộ dữ liệu để truyền vào cho tham số
    data = (id_nha_cung_cap,)
    #sử dụng params để truyền giá trị cho tham số trong Execute
    results = cursor.execute(lenh, params=data)
    #xuất kết quả
    for row in cursor:
        print(row)

In [22]:
id_ncc = input('Nhập mã nhà cung cấp: ')
doc_san_pham_theo_nha_cung_cap(id_ncc, cursor)

Nhập mã nhà cung cấp: 6
(13, 'Product POXFU', 6, 8, 6.0, 0)
(14, 'Product PWCJB', 6, 7, 23.25, 0)
(15, 'Product KSZOI', 6, 2, 15.5, 0)


## Thực hiện nhiều truy vấn đồng thời

In [31]:
lenh = '''
    Select SupplierId, Companyname, Contactname, Address from Suppliers where country = 'Germany';
    Select CustId, Companyname, Contactname, Address from Customers where country = 'Germany';
'''

In [34]:
#thực thi câu lệnh, sử dụng thuộc tính multi=True để khai báo nhiều câu lệnh 
results = cursor.execute(lenh, multi=True)

In [35]:
#câu lệnh gồm nhiều truy vấn --> Kết quả được lưu trong results
#mỗi lệnh truy vấn sẽ trả về 1 bộ dữ liệu
i = 0
for kq in results:  #duyệt qua từng bộ kết quả
    i+=1
    print('\nBộ kết quả thứ:', i)
    for row in kq:  #duyệt qua từng dòng trong bộ kết quả
        print(row)


Bộ kết quả thứ: 1
(11, 'Supplier ZPYVS', 'Jain, Mukesh', 'Tiergartenstraße 3456')
(12, 'Supplier SVIYA', 'Regev, Barak', 'Bogenallee 9012')
(13, 'Supplier TEGSC', 'Brehm, Peter', 'Frahmredder 3456')

Bộ kết quả thứ: 2
(1, 'Customer NRZBB', 'Allen, Michael', 'Obere Str. 0123')
(6, 'Customer XHXJV', 'Poland, Carole', 'Forsterstr. 7890')
(17, 'Customer FEVNN', 'Jones, TiAnna', 'Walserweg 4567')
(25, 'Customer AZJED', 'Carlson, Jason', 'Berliner Platz 9012')
(39, 'Customer GLLAG', 'Song, Lolan', 'Maubelstr. 8901')
(44, 'Customer OXFRU', 'Louverdis, George', 'Magazinweg 8901')
(52, 'Customer PZNLA', 'Dupont-Roc, Patrice', 'Heerstr. 4567')
(56, 'Customer QNIVZ', 'Marinova, Nadejda', 'Mehrheimerstr. 9012')
(63, 'Customer IRRVL', 'Veronesi, Giorgio', 'Taucherstraße 1234')
(79, 'Customer FAPSM', 'Wickham, Jim', 'Luisenstr. 0123')
(86, 'Customer SNXOJ', 'Syamala, Manoj', 'Adenauerallee 7890')


## Thực hiện truy vấn với ExecuteMany
##### ví dụ: Cập nhật đơn giá cho các sản phẩm theo yêu cầu:
    - Nếu Mã Nhà cung cấp là 1 và sản phẩm là 'Product IMEHJ' thì đơn giá tăng thêm 5
    - Nếu Mã Nhà cung cấp là 2 và sản phẩm là 'Product EPEIM' thì đơn giá tăng thêm 6
    - Nếu Mã Nhà cung cấp là 3 và sản phẩm là 'Product HMLNI' thì đơn giá tăng thêm 10

In [37]:
sql = '''
update products
set unitprice = unitprice + %s
where productname = %s and SupplierId = %s
'''
datas = [
    (5, 'Product IMEHJ', 1),
    (6, 'Product EPEIM', 2),
    (10, 'Product HMLNI', 3)
]
for data in datas:
    cursor.execute(sql, data)

In [38]:
#Đặt tên cho tham số --> truyền giá trị theo tên thay vì theo thứ tự
sql = '''
update products
set unitprice = unitprice + %(giatang)s
where productname = %(tensp)s and SupplierId = %(idnhacc)s
'''
datas = [
    {'giatang': 5, 'tensp': 'Product IMEHJ', 'idnhacc': 1},
    {'tensp': 'Product EPEIM', 'giatang': 6, 'idnhacc': 2},
    {'tensp': 'Product HMLNI', 'idnhacc': 3, 'giatang': 10}
]
for data in datas:
    cursor.execute(sql, data)  # dùng execute --> thực hiện 3 lần

In [39]:
sql = '''
update products
set unitprice = unitprice + %(giatang)s
where productname = %(tensp)s and SupplierId = %(idnhacc)s
'''
datas = [
    {'giatang': 5, 'tensp': 'Product IMEHJ', 'idnhacc': 1},
    {'tensp': 'Product EPEIM', 'giatang': 6, 'idnhacc': 2},
    {'tensp': 'Product HMLNI', 'idnhacc': 3, 'giatang': 10}
]

cursor.executemany(sql, datas)

In [42]:
cursor.close()
ketnoi.close()